## Algorithm

Here we are going to solve some classic games using our algorithm. The games were taken from textbooks and game theory classes that can be found on the internet. For simplicity, we condense the algorithm into a single function called "solve_game". If you would like to see the resolution step by step, we have also included a file.

In [37]:
using SymPy

# equations
function s_matrix(x) #function to generate strategy matrix
    S = vec(collect(Base.product(reverse(x)...))) #artesian product
    #transform set of strategies into matrix
    S_M = []
    for i = 1:length(S[1])
        d = [y[i] for y in S]
        if i == 1 S_M = [d; ] else S_M = [S_M d] end
    end
    S_M = reverse(S_M, dims = 2)
    return S_M
end

# the sum of the odds for each player must equal one 
function sum_prob(x)
    eqn = []
    for i = 1:size(x, 2) 
        f = Eq(sum(unique(x[:,i])),1)
        if i == 1 eqn = [f; ] else eqn = [eqn f] end
    end
    return eqn
end

#Hadamard Product
function hadamard_product(x)
    ncol = size(x, 2)
    d = []
    for i = 1:(ncol - 1)
        if i == 1 d = (x[:,i] .* x[:,(i+1)]) else d = (x[:,i+1] .* d) end   
    end    
    return d
end



function foc(eq1,eq2,P)
    eq2_aux = []
    eq3 = Vector{Sym}()
    for pl in 1:size(P,2) # pl = jogador
        var = unique(P[:,pl])
        for i in 1:(size(var)[1]-1)
            for j in var 
                if j != var[i] 
                    eq2_aux = eq2[pl].subs.(j,solve(eq1[pl],j))
                    eq = Eq(diff.(eq2_aux[1],var[i]),0)
                    append!(eq3, eq)
                end
            end
        end
    end
    return eq3
end


foc (generic function with 1 method)

## Example: Battle of the sexes 
https://saylordotorg.github.io/text_introduction-to-economic-analysis/s17-03-mixed-strategies.html

In [38]:
#strategy matrix
a1 = [Sym("a_$i^$j") for i=1 for j=["1","2"]]
a2 = [Sym("a_$i^$j") for i=2 for j=["1","2"]]
A = [a1 , a2]
S = s_matrix(A) 

#Probability matrix 
p1 = [symbols("alpha_$i^$j", real = true, positive = true) for i=1 for j=["1","2"]]
p2 = [symbols("alpha_$i^$j", real = true, positive = true) for i=2 for j=["1","2"]]
P = [p1 , p2]
P = s_matrix(P) 

#payoff matrix
U = [3 2;1 1;0 0;2 3]

4×2 Matrix{Int64}:
 3  2
 1  1
 0  0
 2  3

In [39]:
eq1 = sum_prob(P) #first set of equations
S_P = hadamard_product(P) #hadamard product
eq2 = S_P.T * U #spected payoff
eq3 = foc(eq1,eq2,P) #second set of equations
eq4 = append!(vec(eq1), eq3) # join first and second set of equations 
vars = unique(P) #system variables 
sol = solve(eq4,vars) #solve sistem
sol = [Eq(var[1],var[2]) for var in sol] #print result

4-element Vector{Sym}:
 α¹₂ = 1/4
 α¹₁ = 3/4
 α²₁ = 1/4
 α²₂ = 3/4

From now on, we're going to condense these steps into just one function caled: "solve_game". This new function depends only on the probability matrix and the payoff matrix. 

In [40]:
function solve_game(P,U)
    eq1 = sum_prob(P) #first set of equations
    S_P = hadamard_product(P) #hadamard product
    eq2 = S_P.T * U #spected payoff
    eq3 = foc(eq1,eq2,P) #second set of equations
    eq4 = append!(vec(eq1), eq3) # join first and second set of equations 
    vars = unique(P) #system variables 
    sol = solve(eq4,vars) #solve sistem
    sol = [Eq(var[1],var[2]) for var in sol] #print result
    return sol
end

solve_game (generic function with 1 method)

## Example: Matching pennies

https://saylordotorg.github.io/text_introduction-to-economic-analysis/s17-03-mixed-strategies.html

In [41]:
U = [1 -1;-1 1;-1 1;1 -1]
solve_game(P,U)

4-element Vector{Sym}:
 α¹₂ = 1/2
 α¹₁ = 1/2
 α²₁ = 1/2
 α²₂ = 1/2

## Example: Chicken

https://saylordotorg.github.io/text_introduction-to-economic-analysis/s17-03-mixed-strategies.html

In [42]:
U = [0 0;-1 1;1 -1;-4 -4]
solve_game(P,U)

4-element Vector{Sym}:
 α¹₂ = 3/4
 α¹₁ = 3/4
 α²₁ = 1/4
 α²₂ = 1/4

## Example: Welfare Game
Rasmusen (pp. 67)

In [43]:
U = [3 2;-1 3;-1 1;0 0]
solve_game(P,U)

4-element Vector{Sym}:
 α¹₂ = 1/5
 α¹₁ = 1/2
 α²₁ = 1/2
 α²₂ = 4/5

## Example: Grab the Dollar

Rasmusen (pp. 73)

In [28]:
U = [-1 -1;1 0;0 1;0 0]
solve_game(P,U)

4-element Vector{Sym}:
 α¹₂ = 1/2
 α¹₁ = 1/2
 α²₁ = 1/2
 α²₂ = 1/2

## Example: The Civic Duty Game

Rasmusen (pp. 80)

In [29]:
U = [0 0;10 7;7 10;7 7]
solve_game(P,U)

4-element Vector{Sym}:
 α¹₂ = 3/10
 α¹₁ = 3/10
 α²₁ = 7/10
 α²₂ = 7/10

## Example: The Civic Duty Game (simbolic)

Rasmusen (pp. 77)

In [30]:
a,w,b,x,c,y,d,z = symbols("a,w,b,x,c,y,d,z", real = true)
U = [a w;b x;c y;d z]
solve_game(P,U)

4-element Vector{Sym}:
  Eq(alpha_2^1, (-b + d)/(a - b - c + d))
  Eq(alpha_1^1, (y - z)/(-w + x + y - z))
 Eq(alpha_1^2, (-w + x)/(-w + x + y - z))
   Eq(alpha_2^2, (a - c)/(a - b - c + d))

## Example: Auditing Game I (simbolic)
Rasmusen (pp. 82)

In [31]:
c,f = symbols("c,f", real = true)
U = [4-c -f;4-c -1;0 0;4 -1]
solve_game(P,U)

4-element Vector{Sym}:
       Eq(alpha_2^1, c/4)
       Eq(alpha_1^1, 1/f)
 Eq(alpha_1^2, (f - 1)/f)
   Eq(alpha_2^2, 1 - c/4)

## Example: Penalty kicks in soccer (three actions)

https://felixmunozgarcia.files.wordpress.com/2017/08/slides_71.pdf

In [32]:
#strategy matrix
Goalkeeper = [Sym("a_$i^$j") for i="G" for j=["l","c","r"]]
Kicker = [Sym("a_$i^$j") for i="K" for j=["l","c","r"]]
A = [Goalkeeper , Kicker]
S = s_matrix(A) 

#Probability matrix 
Goalkeeper_p = [symbols("alpha_$i^$j", real = true, positive = true) for i="G" for j=["l","c","r"]]
Kicker_p = [symbols("alpha_$i^$j", real = true, positive = true) for i="K" for j=["l","c","r"]]
P = [Goalkeeper_p , Kicker_p]
P = s_matrix(P) 

9×2 Matrix{Sym}:
 α_G__l  α_K__l
 α_G__l  α_K__c
 α_G__l  α_K__r
 α_G__c  α_K__l
 α_G__c  α_K__c
 α_G__c  α_K__r
 α_G__r  α_K__l
 α_G__r  α_K__c
 α_G__r  α_K__r

In [33]:
U = [0.65 0.35;0.95 0.05;0.95 0.05;0.95 0.05; 0 1;0.95 0.05;0.95 0.05;0.95 0.05;0.65 0.35]

9×2 Matrix{Float64}:
 0.65  0.35
 0.95  0.05
 0.95  0.05
 0.95  0.05
 0.0   1.0
 0.95  0.05
 0.95  0.05
 0.95  0.05
 0.65  0.35

In [34]:
solve_game(P,U)

6-element Vector{Sym}:
 α_G__r = 0.431818181818182
 α_G__l = 0.431818181818182
 α_G__c = 0.136363636363636
 α_K__c = 0.136363636363636
 α_K__l = 0.431818181818182
 α_K__r = 0.431818181818182

## Example: three players

https://sites.duke.edu/niou/files/2011/05/Lecture-3.pdf

In [35]:
#Probability matrix 
p1 = [symbols("alpha_$i^$j", real = true, positive = true) for i=1 for j=["a","b"]]
p2 = [symbols("alpha_$i^$j", real = true, positive = true) for i=2 for j=["a","b"]]
p3 = [symbols("alpha_$i^$j", real = true, positive = true) for i=3 for j=["a","b"]]
P = [p1 , p2 , p3]
P = s_matrix(P) 

#payoff matrix
U = [1 1 1; 0 0 0; 0 0 0;0 0 0;0 0 0;0 0 0;0 0 0;4 4 4]

#solve
eq1 = sum_prob(P) #first set of equations
S_P = hadamard_product(P) #hadamard product
eq2 = S_P.T * U #spected payoff
eq3 = foc(eq1,eq2,P) #second set of equations
eq4 = append!(vec(eq1), eq3) # join first and second set of equations 
vars = unique(P) #system variables 
sol = solve(eq4,vars) #solve sistem
Eq.(vec(vars),sol[1])#print result

6-element Vector{Sym}:
 α_1__a = 2/3
 α_1__b = 1/3
 α_2__a = 2/3
 α_2__b = 1/3
 α_3__a = 2/3
 α_3__b = 1/3

## Example: three players
https://felixmunozgarcia.files.wordpress.com/2017/08/slides_71.pdf

In [36]:
#strategy matrix
Beth = [Sym("a_$i^$j") for i="B" for j=["f","b"]]
Tommy = [Sym("a_$i^$j") for i="T" for j=["f","b"]]
Jason = [Sym("a_$i^$j") for i="J" for j=["f","b"]]
A = [Beth , Tommy, Jason]
S = s_matrix(A) 

#Probability matrix 
Beth_p = [symbols("alpha_$i^$j", real = true, positive = true) for i="B" for j=["f","b"]]
Tommy_p = [symbols("alpha_$i^$j", real = true, positive = true) for i="T" for j=["f","b"]]
Jason_p = [symbols("alpha_$i^$j", real = true, positive = true) for i="J" for j=["f","b"]]
P = [Beth_p , Tommy_p, Jason_p]
P = s_matrix(P) 

#payoff matrix
U = [0 0 0; 3 3 -2;-4 1 2;1 -4 2;1 -4 2;-4 1 2; 2 2 -2; 0 0 0]

#solve
eq1 = sum_prob(P) #first set of equations
S_P = hadamard_product(P) #hadamard product
eq2 = S_P.T * U #spected payoff
eq3 = foc(eq1,eq2,P) #second set of equations
eq4 = append!(vec(eq1), eq3) # join first and second set of equations 
vars = unique(P) #system variables 
sol = solve(eq4,vars) #solve sistem
Eq.(vec(vars),sol[1])#print result

6-element Vector{Sym}:
  α_B__f = 1/2
  α_B__b = 1/2
  α_T__f = 1/2
  α_T__b = 1/2
 α_J__f = 8/15
 α_J__b = 7/15